In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%ls

drive/  sample_data/


In [4]:
test = pd.read_csv("/content/drive/MyDrive/7313Project/test_with_all.csv")
train = pd.read_csv("/content/drive/MyDrive/7313Project/train_with_all.csv")
testn = pd.read_csv("/content/drive/MyDrive/7313Project/test_with_all.csv")
target = pd.read_csv("/content/drive/MyDrive/7313Project/target.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
train.drop(train.iloc[:, [0,1]], axis=1, inplace=True)
test.drop(test.iloc[:, [0,1]], axis=1, inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [7]:
train['is_cup'] = labelencoder.fit_transform(train['is_cup'])
test['is_cup'] = labelencoder.fit_transform(test['is_cup'])
target['target'].replace({"home": 0, "draw": 1, "away":2}, inplace=True)
train.drop(train.filter(regex='recorded_matches').columns, axis=1, inplace = True)
test.drop(test.filter(regex='recorded_matches').columns, axis=1, inplace = True)
train_x = train.copy()
train_y = target['target'].copy()
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [8]:
!pip install --upgrade xgboost

     |████████████████████████████████| 193.7 MB 33 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [9]:
from sklearn.preprocessing import StandardScaler
#fit scaler on training data
norm = StandardScaler().fit(train_x)
#transform training data
xtrain_norm = norm.transform(train_x)
#transform testing data
test_norm = norm.transform(test)

In [10]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# import xgboost as xgb

# cv_acc = cross_val_score(estimator=xgb.XGBClassifier(),
#                          X=xtrain_norm,
#                          y=train_y,
#                          cv=5,
#                          n_jobs=5)

# print('Kfold Accuracy: %.2f%%' % (np.mean(cv_acc)*100))

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold
import xgboost as xgb
test_preds_1 = []
test_preds_2 = []

kf = StratifiedKFold(n_splits=5, random_state=123, shuffle=True) # KFold
i=0
for fold, (train_idx, test_idx) in enumerate(kf.split(xtrain_norm, train_y)):
    i=i+1
    print(i)
    X_train, X_test = xtrain_norm[train_idx], xtrain_norm[test_idx]
    y_train, y_test = train_y.iloc[train_idx], train_y.iloc[test_idx]
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    train_acc = np.mean(y_pred == y_test)*100
    confmat = confusion_matrix(y_pred, y_test)
    print(train_acc)
    print(confmat)
    test_preds_1.append(model.predict_proba(xtrain_norm).squeeze())
    test_preds_2.append(model.predict_proba(test_norm).squeeze())

1
2
3
4
5


In [14]:
from mlxtend.evaluate import confusion_matrix
predictions = sum(test_preds_1)/5
classes = np.argmax(predictions, axis = 1)
print(classes)
y_true = train_y.squeeze()
confmat = confusion_matrix(y_true, classes)

print(confmat)

[2 0 2 ... 1 0 0]
[[42421   259  5433]
 [15366  5814  6471]
 [12980   365 21829]]


In [15]:
train_acc = np.mean(classes == y_true)*100
train_acc

63.15599704339361

In [16]:
prediction_1 = sum(test_preds_2)/5
prediction_1

array([[0.42552224, 0.32397112, 0.25050658],
       [0.36555204, 0.31960824, 0.31483975],
       [0.3499853 , 0.3561719 , 0.2938428 ],
       ...,
       [0.13650794, 0.24840207, 0.61509   ],
       [0.5540945 , 0.28754282, 0.15836272],
       [0.32603976, 0.3033595 , 0.37060076]], dtype=float32)

In [17]:
submission = pd.concat([pd.DataFrame(testn['id']), pd.DataFrame(prediction_1, columns=['home', 'draw', 'away'])[['home', 'draw', 'away']]], axis=1)
submission.to_csv('/content/drive/MyDrive/7313Project/submission_xgb.csv', index=False)